In [2]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [3]:
%cd slt

/content/slt


In [4]:
!chmod +x data/download.sh
!data/download.sh

--2021-01-03 20:37:50--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  17.1MB/s    in 1m 48s  

2021-01-03 20:39:39 (17.1 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-03 20:39:39--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  16.8MB/s    in 8.6s    

2021-01-03 20:39:48 (14.6 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [6]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 12.2MB/s 
     |████████████████████████████████| 112kB 12.0MB/s 
     |████████████████████████████████| 163kB 22.6MB/s 
     |████████████████████████████████| 399kB 28.6MB/s 
     |████████████████████████████████| 2.3MB 34.7MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 2.4MB 59.3MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 296kB 62.9MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 13.1MB 44.0MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 20.2MB 365kB/s 
     |█

In [9]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!cp -r ../drive/MyDrive/slt/keypoints/dev_features.zip data/dev_features.zip
!cp -r ../drive/MyDrive/slt/keypoints/train_features.zip data/train_features.zip
!cp -r ../drive/MyDrive/slt/keypoints/test_features.zip data/test_features.zip

!unzip -q data/dev_features.zip -d data
!unzip -q data/train_features.zip -d data
!unzip -q data/test_features.zip -d data

In [10]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

for s in ['train', 'dev', 'test']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)
  for i, src in tqdm(enumerate(src_features)):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)
  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

7096it [00:33, 208.97it/s]
519it [00:06, 81.84it/s]
642it [00:07, 87.39it/s]


In [11]:
!ls data/PHOENIX2014T

phoenix14t.pami0.dev   phoenix14t.pami0.train  phoenix14t.pami1.test
phoenix14t.pami0.test  phoenix14t.pami1.dev    phoenix14t.pami1.train


In [12]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 1024]
list_doc['training']['recognition_loss_weight'] = 20

In [14]:
with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [15]:
!python -m signjoey train configs/sign.yaml

2021-01-03 20:55:30.114262: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-03 20:55:36.710866: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-03 20:55:36.729984: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-03 20:55:36.791857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-03 20:55:36.792561: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-01-03 20:55:36.792621: I tensorflow/stream_executor/platform/defau